In [12]:
from Environments.BaseMarket import TestEnv
from Environments.LimitOrderMarket import LimitMarket
from Environments.wrappers.reward_wrapper import CuriosityWrapper

import numpy as np
import matplotlib.pyplot as plt
import ray
import gym
from ray.tune.registry import register_env
from ray.rllib.agents.ppo.ddppo import DEFAULT_CONFIG
from ray.rllib.agents.ppo.ddppo import DDPPOTrainer
import os
import gc
import torch 
import pandas as pd
gc.collect()

0

In [13]:
try:
    ray.init(num_gpus = 1, num_cpus = 4)
except:
    ray.shutdown()
    ray.init(num_gpus = 1, num_cpus = 4)

2021-06-09 13:44:10,580	INFO services.py:1272 -- View the Ray dashboard at http://127.0.0.1:8265


In [14]:
trainer_config = DEFAULT_CONFIG.copy()

In [15]:
trainer_config['model']

{'_use_default_native_models': False,
 'fcnet_hiddens': [1024, 1024],
 'fcnet_activation': 'tanh',
 'conv_filters': None,
 'conv_activation': 'relu',
 'post_fcnet_hiddens': [],
 'post_fcnet_activation': 'relu',
 'free_log_std': False,
 'no_final_linear': False,
 'vf_share_layers': True,
 'use_lstm': False,
 'max_seq_len': 20,
 'lstm_cell_size': 512,
 'lstm_use_prev_action': False,
 'lstm_use_prev_reward': False,
 '_time_major': False,
 'use_attention': False,
 'attention_num_transformer_units': 1,
 'attention_dim': 64,
 'attention_num_heads': 1,
 'attention_head_dim': 32,
 'attention_memory_inference': 50,
 'attention_memory_training': 50,
 'attention_position_wise_mlp_dim': 32,
 'attention_init_gru_gate_bias': 2.0,
 'attention_use_n_prev_actions': 0,
 'attention_use_n_prev_rewards': 0,
 'num_framestacks': 'auto',
 'dim': 84,
 'grayscale': False,
 'zero_mean': True,
 'custom_model': None,
 'custom_model_config': {},
 'custom_action_dist': None,
 'custom_preprocessor': None,
 'lstm_use_

In [16]:
trainer_config['model']['use_lstm'] = False
trainer_config['model']['lstm_cell_size'] = 512
trainer_config['num_gpus'] = 0
trainer_config['num_gpus_per_worker'] = 1
trainer_config['num_envs_per_worker'] = 1
trainer_config['gamma'] = 0
# trainer_config['entropy_coeff'] = 0
trainer_config['framework'] = 'torch'
trainer_config['num_workers'] = 1
trainer_config['horizon'] = 1000
trainer_config['rollout_fragment_length'] = 1000
trainer_config['model']['framestack'] = False
trainer_config['model']['fcnet_hiddens'] = [1024, 1024]
conf = {'data': 'Data/indicator_dataset/',
        'starting_money': 1000,
        'starting_stocks': 0,
        'episode_length': 1000,
        'commission': 0.0025,
        'state_orders_num': 10,
        'max_horizon' : 100,
        'curiosity_reward' : 0
        }
trainer_config['env_config'] = conf
# trainer_config['entropy_coeff_schedule'] = 1000

In [17]:
def curiosity_env_create(env_config):
    return TestEnv(LimitMarket(env_config))

In [18]:
register_env('CuriosityLimitMarket', curiosity_env_create)

In [21]:
trainer = DQNTrainer(trainer_config, env = 'CuriosityLimitMarket')

(pid=10740) 2021-06-09 13:44:33,479	ERROR worker.py:409 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=10740, ip=192.168.1.100)
(pid=10740)   File "python\ray\_raylet.pyx", line 501, in ray._raylet.execute_task
(pid=10740)   File "python\ray\_raylet.pyx", line 451, in ray._raylet.execute_task.function_executor
(pid=10740)   File "D:\Program_files\Anaconda\envs\ray\lib\site-packages\ray\_private\function_manager.py", line 563, in actor_method_executor
(pid=10740)     return method(__ray_actor, *args, **kwargs)
(pid=10740)   File "D:\Program_files\Anaconda\envs\ray\lib\site-packages\ray\rllib\evaluation\rollout_worker.py", line 536, in __init__
(pid=10740)     self.policy_map, self.preprocessors = self._build_policy_map(
(pid=10740)   File "D:\Program_files\Anaconda\envs\ray\lib\site-packages\ray\rllib\evaluation\rollout_worker.py", line 1196, in _build_policy_map
(pid=10740)     policy_map[name] = 

(pid=10740) Stream name: -29d8a821-9702-4a49-a3a6-ec06cc9e26c7


RayActorError: The actor died because of an error raised in its creation task, [36mray::RolloutWorker.__init__()[39m (pid=10740, ip=192.168.1.100)
  File "python\ray\_raylet.pyx", line 501, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 451, in ray._raylet.execute_task.function_executor
  File "D:\Program_files\Anaconda\envs\ray\lib\site-packages\ray\_private\function_manager.py", line 563, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "D:\Program_files\Anaconda\envs\ray\lib\site-packages\ray\rllib\evaluation\rollout_worker.py", line 536, in __init__
    self.policy_map, self.preprocessors = self._build_policy_map(
  File "D:\Program_files\Anaconda\envs\ray\lib\site-packages\ray\rllib\evaluation\rollout_worker.py", line 1196, in _build_policy_map
    policy_map[name] = cls(obs_space, act_space, merged_conf)
  File "D:\Program_files\Anaconda\envs\ray\lib\site-packages\ray\rllib\policy\policy_template.py", line 236, in __init__
    s

In [ ]:
best_reward = -np.inf
trainer.save()
hall_of_fame = [0]
last_checkpoint = 0

In [ ]:
%%time
for i in range(100):
    print("Training iteration {}...".format(i))
    results = trainer.train()
    this_reward = results['episode_reward_max']
    if this_reward > best_reward:
        best_reward = this_reward
        trainer.save()
        path = trainer.logdir + 'checkpoint_{0}/checkpoint-{0}'.format(last_checkpoint)
        os.remove(path)
        last_checkpoint = i + 1
        hall_of_fame.append(i+1)
        print('New best reward')
        print(best_reward)
    if i % 10 == 0:
        print('Best Reward So Far')
        print(best_reward)      

In [ ]:
path = trainer.logdir + 'checkpoint_{0}/checkpoint-{0}'.format(hall_of_fame[-1])

In [ ]:
training = pd.read_csv(trainer.logdir + 'progress.csv')
plt.plot(training['episode_reward_mean'])

In [ ]:
env = LimitMarket(conf)
obs = env.reset()

done = False
cumulative_reward = 0
prices = []
assets = []
actions = []
states = [obs]
rewards = []
hidden = [torch.zeros(512),torch.zeros(512)]
infos = []
while not done:
    action, hidden, info = trainer.compute_action(obs, hidden)
    obs, reward, done, results = env.step(action)
    cumulative_reward += reward
    rewards.append(reward)
    actions.append(action)
    assets.append(results['assets'])
    prices.append(results['current_price'])
    states.append(obs)
    infos.append(info)
    if i % 100 == 0:
        print('Step: {}/{}'.format(i, 200))
print("Cumulative reward you've received is: {}. Congratulations!".format(cumulative_reward))
print("Asset_Gain {}".format(assets[-1] -assets[0]))

In [ ]:
pure_actions = []
for action in actions:
    pure_actions.append(action[0])
    
actions = pure_actions

In [ ]:
buy10 = np.ma.masked_where(np.array(actions) != 0, prices)
buy20 = np.ma.masked_where(np.array(actions) != 1, prices)
buy50 = np.ma.masked_where(np.array(actions) != 2, prices)
sell10 = np.ma.masked_where(np.array(actions) != 3, prices)
sell20 = np.ma.masked_where(np.array(actions) != 4, prices)
sell50 = np.ma.masked_where(np.array(actions) != 5, prices)
hold = np.ma.masked_where(np.array(actions) != 6, prices)

# plt.plot(prices, marker = '', markersize = 0.5, markevery = np.where(np.array(actions) == 6, True, False))
# plt.figure(figsize = (20, 15))
# plt.plot(buy10, c = 'turquoise', linewidth = 0.6)
# plt.plot(buy20, c = 'lime', linewidth = 0.6)
# plt.plot(buy50, c = 'green', linewidth = 0.6)
# plt.plot(hold, c = 'blue', linewidth = 0.6)
graph_prices = prices[::10][:500]
graph_actions = actions[::10][:500]
colors = ['r', 'r', 'r', 'g', 'g', 'g', 'b']
fig = plt.figure(figsize = (10, 6))
plt.scatter(range(len(graph_prices)), graph_prices, s=1, color = np.array(colors)[graph_actions])

In [ ]:
plt.plot(assets)

In [ ]:
plt.plot(prices)

In [ ]:
obs = env.test()

done = False
cumulative_reward = 0
prices = []
assets = []
actions = []
states = [obs]
rewards = []
hidden = [torch.zeros(512),torch.zeros(512)]
infos = []
market_beaters = []
while not done:
    action, hidden, info = trainer.compute_action(obs, hidden)
    obs, reward, done, results = env.step(action)
    cumulative_reward += reward
    rewards.append(reward)
    actions.append(action)
    assets.append(results['assets'])
    prices.append(results['current_price'])
    states.append(obs)
    infos.append(info)
    market_beaters.append(results['market_beater'])
print("Cumulative reward you've received is: {}. Congratulations!".format(cumulative_reward))
print("Asset_Gain {}".format(assets[-1] -assets[0]))

In [ ]:
plt.plot(market_beaters)

In [ ]:
pure_actions = []
for action in actions:
    print(action)

In [ ]:
buy10 = np.ma.masked_where(np.array(actions) == 0, prices)
buy20 = np.ma.masked_where(np.array(actions) == 1, prices)
buy50 = np.ma.masked_where(np.array(actions) == 2, prices)
hold = np.ma.masked_where(np.array(actions) == 3, prices)

plt.plot(buy10, c = 'turquoise')
plt.plot(buy20, c = 'lime')
plt.plot(buy50, c = 'green')
plt.plot(hold, c = 'blue')

In [ ]:
plt.plot(assets)

In [ ]:
plt.plot(prices)

In [ ]:
market_beaters[-1]

In [ ]:
from sklearn import preprocessing

In [ ]:
prices_norm = preprocessing.normalize(np.array(prices).reshape(-1,1))
assets_norm = preprocessing.normalize(np.array(assets).reshape(-1,1))

In [ ]:
assets_norm-prices_norm